In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score

from code.utils import *

## Data

In [2]:
train = data(features('data/training/'), labels('data/training-class'))

In [3]:
test = data(features('data/test/'), labels('data/test-class'))

## Featurize

**Note: need to have the same vocabulary**

Tokenization is important here

In [4]:
tfidf = TfidfVectorizer()

In [5]:
X_train = tfidf.fit_transform(train.text.values)
y_train = train.label.values

In [6]:
X_train.shape

(2638, 36447)

In [7]:
X_test = tfidf.fit_transform(test.text.values)
y_test = test.label.values

In [8]:
X_test.shape

(1595, 28365)

## Model

In [9]:
clf = LinearSVC()

In [10]:
clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

**Note: predict on the test set**

Currently predicting on the training set because of class imbalance&mdash;all of the predicted labels on the test set were zero

In [11]:
y_hat = clf.predict(X_train)

In [12]:
accuracy_score(y_train, y_hat)

1.0

In [13]:
f1_score(y_train, y_hat)

1.0